In [1]:
import pandas as pd
import numpy as np
import pickle

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

#import library requirements
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

# **Load Model**

In [2]:
# Load the trained model and preprocessing pipeline using pickle
filename = 'final_model_gbc_bank_marketing_campaign.sav'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

# **Create Observation**

In [3]:
new_cust = pd.DataFrame({
    'age': [56,24],
    'housing' : ['no','yes'],
    'loan' : ['no','yes'],
    'contact' : ['unknown','cellular'],
    'balance' : [1000,488],
    'job' : ['admin','management'],
    'month' : ['jan','apr'],
    'campaign' : [2,1],
    'pdays' : [-1,-1],
    'poutcome' : ['failure','success']
    })
new_cust

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome
0,56,no,no,unknown,1000,admin,jan,2,-1,failure
1,24,yes,yes,cellular,488,management,apr,1,-1,success


In [6]:
#new_cust2 = pd.DataFrame({
    'age': [24],
    'housing' : ['no'],
    'loan' : ['no'],
    'contact' : ['cellular'],
    'balance' : [448],
    'job' : ['management'],
    'month' : ['apr'],
    'campaign' : [1],
    'pdays' : [-1],
    'poutcome' : ['unknown']
    })
new_cust2

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome
0,24,no,no,cellular,448,management,apr,1,-1,unknown


In [4]:
df = pd.read_csv("data_bank_marketing_campaign.csv")
df.head()

,age,job,balance,housing,loan,contact,month,campaign,pdays,poutcome,deposit
0,55,admin.,1662,no,no,cellular,jun,2,-1,unknown,yes
1,39,self-employed,-3058,yes,yes,cellular,apr,3,-1,unknown,yes
2,51,admin.,3025,no,no,cellular,may,1,352,other,yes
3,38,services,-87,yes,no,cellular,may,1,-1,unknown,no
4,36,housemaid,205,yes,no,telephone,nov,4,-1,unknown,no


In [5]:
# Separate the features and the target variable
X = df.drop('deposit', axis=1)
y = df['deposit']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    stratify=y,
    test_size=0.2,
    random_state=0
)

# Define the preprocessing steps with handle_unknown='ignore'
transformer = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['job', 'housing', 'loan', 'contact', 'month', 'poutcome'])],
    remainder='passthrough'
)

# Create a pipeline that includes the transformer, scaler, and model
pipeline = Pipeline(steps=[
    ('preprocessor', transformer),
    ('scaler', StandardScaler(with_mean=False)),
    ('classifier', GradientBoostingClassifier(random_state=0))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Save the entire pipeline (including the preprocessor and scaler)
filename = 'final_model_gbc_bank_marketing_campaign.sav'
with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)

print("Model and preprocessing pipeline saved to 'final_model_gbc_bank_marketing_campaign.sav'.")

Model and preprocessing pipeline saved to 'final_model_gbc_bank_marketing_campaign.sav'.


## **Upload GCS**

In [6]:
import os
## set up authentication using services account 
# Authenticate using service account
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../zaki-013/sa-development.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../zaki-013/sa-development.json"

In [7]:
# Load data from Google Biquery to local machine

project_id = 'dti-ds'
dataset_id = 'zaki_dataset_013'
table_id = 'x_bank_marketing_campaign'
region = 'us-central1'
bucket_name = 'zaki_gcs_013'
blob_name = 'x_bank_marketing_campaign.csv'
model_name = 'final_model_gbc_bank_marketing_campaign.sav'

In [8]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project='dti-ds')
    bucket = storage_client.get_bucket('zaki_gcs_013') # Add bucket name
    blob_model = bucket.blob('Bank-Marketing-Campaign/final_model_gbc_bank_marketing_campaign.sav')
    blob_model.upload_from_filename('final_model_gbc_bank_marketing_campaign.sav')
  
    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


# **Load Data From BigQuerry**

In [9]:
#load data from BQ
from google.cloud import bigquery
## using bigquery client 
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df_cloud= query_job.result().to_dataframe()

df_cloud = df_cloud.loc[:, new_cust.columns]
df_cloud

/home/ammarmuzacky/miniconda3/envs/zaki-bmc/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome
0,18,False,False,cellular,608,student,aug,1,-1,unknown
1,18,False,False,cellular,108,student,aug,1,-1,unknown
2,18,False,False,cellular,108,student,feb,1,183,success
3,18,False,False,cellular,348,student,may,4,-1,unknown
4,18,False,False,cellular,608,student,nov,1,93,success
...,...,...,...,...,...,...,...,...,...,...
7808,90,False,False,telephone,712,retired,mar,1,-1,unknown
7809,92,False,False,cellular,775,retired,jan,4,96,success
7810,93,False,False,cellular,775,retired,aug,2,13,success
7811,93,False,False,cellular,775,retired,jul,2,177,success


In [10]:
df_cloud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813 entries, 0 to 7812
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       7813 non-null   Int64  
 1   housing   7813 non-null   boolean
 2   loan      7813 non-null   boolean
 3   contact   7813 non-null   object 
 4   balance   7813 non-null   Int64  
 5   job       7813 non-null   object 
 6   month     7813 non-null   object 
 7   campaign  7813 non-null   Int64  
 8   pdays     7813 non-null   Int64  
 9   poutcome  7813 non-null   object 
dtypes: Int64(4), boolean(2), object(4)
memory usage: 549.5+ KB


In [11]:
df_cloud['housing'] = df_cloud['housing'].astype(str).map({'True': 'yes', 'False': 'no'})


In [12]:
df_cloud['loan'] = df_cloud['loan'].astype(str).map({'True': 'yes', 'False': 'no'})

In [13]:
df_cloud.head()

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome
0,18,no,no,cellular,608,student,aug,1,-1,unknown
1,18,no,no,cellular,108,student,aug,1,-1,unknown
2,18,no,no,cellular,108,student,feb,1,183,success
3,18,no,no,cellular,348,student,may,4,-1,unknown
4,18,no,no,cellular,608,student,nov,1,93,success


In [14]:
df_cloud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813 entries, 0 to 7812
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   age       7813 non-null   Int64 
 1   housing   7813 non-null   object
 2   loan      7813 non-null   object
 3   contact   7813 non-null   object
 4   balance   7813 non-null   Int64 
 5   job       7813 non-null   object
 6   month     7813 non-null   object
 7   campaign  7813 non-null   Int64 
 8   pdays     7813 non-null   Int64 
 9   poutcome  7813 non-null   object
dtypes: Int64(4), object(6)
memory usage: 641.0+ KB


In [15]:
new_cust

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome
0,56,no,no,unknown,1000,admin,jan,2,-1,failure
1,24,yes,yes,cellular,488,management,apr,1,-1,success


In [16]:
# Load the model
with open('final_model_gbc_bank_marketing_campaign.sav', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud_new_data = loaded_model.predict(new_cust)
y_pred_cloud_new_data

array(['no', 'yes'], dtype=object)

In [17]:
# data observation predicted
new_test_cloud = new_cust.copy()
new_test_cloud['open deposit'] = y_pred_cloud_new_data
new_test_cloud

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome,open deposit
0,56,no,no,unknown,1000,admin,jan,2,-1,failure,no
1,24,yes,yes,cellular,488,management,apr,1,-1,success,yes


In [18]:
# loaded_model.predict(X_test.iloc[3:13])
y_pred_file_cloud = loaded_model.predict(df_cloud)
y_pred_file_cloud[:12]

array(['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes',
       'yes', 'yes', 'no'], dtype=object)

In [19]:
df_cloud['open deposit'] = y_pred_file_cloud
df_cloud

,age,housing,loan,contact,balance,job,month,campaign,pdays,poutcome,open deposit
0,18,no,no,cellular,608,student,aug,1,-1,unknown,yes
1,18,no,no,cellular,108,student,aug,1,-1,unknown,yes
2,18,no,no,cellular,108,student,feb,1,183,success,yes
3,18,no,no,cellular,348,student,may,4,-1,unknown,yes
4,18,no,no,cellular,608,student,nov,1,93,success,yes
...,...,...,...,...,...,...,...,...,...,...,...
7808,90,no,no,telephone,712,retired,mar,1,-1,unknown,yes
7809,92,no,no,cellular,775,retired,jan,4,96,success,yes
7810,93,no,no,cellular,775,retired,aug,2,13,success,yes
7811,93,no,no,cellular,775,retired,jul,2,177,success,yes


****

In [20]:
table_id ='x_unseen_bank_marketing_campaign_predicted'

#construct a BiqQuery client object
client = bigquery.Client()

#Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

df_cloud.columns = ['_'.join(i.split(' ')) for i in df_cloud.columns]

# Load the DtaFrame into the BigQuerry table
job = client.load_table_from_dataframe(df_cloud,table_full_id)

job.result()  
print(f"Loaded {job.output_rows} rows into {table_full_id}")


Loaded 7813 rows into dti-ds.zaki_dataset_013.x_unseen_bank_marketing_campaign_predicted
